In [1]:
import sys, os, time
from solc import compile_source, compile_files, link_code
from ethjsonrpc import EthJsonRpc

print("Using environment in "+sys.prefix)
print("Python version "+sys.version)

Using environment in /Users/emunsing/Documents/Coding/github/energyblockchain/env/bin/..
Python version 2.7.12 (default, Aug 23 2016, 08:54:23) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]


In [33]:
# Initiate connection to ethereum node
#   Requires a node running with an RPC connection available at port 8545
c = EthJsonRpc('127.0.0.1', 8545)
print(c.web3_clientVersion())
print("Block number %s"%c.eth_blockNumber())

EthereumJS TestRPC/v1.0.1/ethereum-js
Block number 9


In [61]:
source = """pragma solidity ^0.4.2;

contract mortal {
    /* Define variable owner of the type address*/
    address owner;

    /* this function is executed at initialization and sets the owner of the contract */
    function mortal() { owner = msg.sender; }

    /* Function to recover the funds on the contract */
    function kill() { if (msg.sender == owner) selfdestruct(owner); }
}

contract greeter is mortal {
    /* define variable greeting of the type string */
    string greeting;
    address greetername;

    /* this runs when the contract is executed */
    function greeter(string _greeting) public {
        greeting = _greeting;
        greetername = msg.sender;
    }
    
    /* main function */
    function greetme() constant returns (string) {
        return greeting;
    }
    
    function originator() constant returns (address ret) {
        return greetername;
    }
    
    function greettwo() constant returns (string) {
        return greeting;
    }

}"""

- *i* participants/nodes
- *t* time periods
- *k* ADMM iteration
- *z* global ADMM estimate at iteration *k*
- *x_i* local variable estimates for node *i*


Pseudocode:
- Variables:
  - Array of *i* addresses (permanent)
  - Current iteration (int)
  - ADMM variable estimates from each participant: *i* entries, each holding ~3n variables
  - Array of people for whom we are still expecting an update this iteration
  - Schedule - array, *t* time steps by *i* nodes
  
- Functions:
  - Initialize (list of addresses):
    - Initialize the global variable estimates
    - Set the schedule to none
    - Set the list of people we want to be everyone
    - Set the iteration to be 1
    - Set the tolerance
  - Recieve update (variable estimate)
    - Check that iteration is current- if not throw
    - Remove message sender from waiting list
    - Store estimate in array
    - If waiting list is now empty, call updater
  - Updater
    - Compute the average value
    - Compute the change in average value
    - If the deviation is above the tolerance
      - Increment the current iteration
    - Else,
      - Save the schedule
      - Set the iteration to throw an error
  - Get schedule:
    - If the schedule is empty, throw
    - If we have a schedule, return the schedule for the message sender

    
Workflow:
- Device submits variable estimate
- Once all devices have submitted estimates, global variable is updated
- Devices poll contract until they see the the iteration number increment
- Process loops until global variable change is less than the tolerance
- Schedule is saved
- Device 

Private: 
- Check if still waiting:
  - index goes until whitelist length
  - 
  - Loop through whitelist
  - if any of those are still waiting, return true
  

In [3]:
source = """pragma solidity ^0.4.2;

contract aggregator {
    address   owner;
    uint8     public iteration;
    address[] public whitelist;
    mapping   (address => bool) public waiting;
    
    
    /* CONSTRUCTOR */
    function aggregator (address[] _whitelist) public{
        whitelist = _whitelist;
        iteration = 1;
        resetWaiting();  // Set the waiting flag to 1 for everybody
    }
        
    function stillWaiting () returns (bool) {
        for (uint8 i=0; i<whitelist.length; i++){
            if (waiting[whitelist[i]]){ return true; }
        }
        return false;
    }
    
    function resetWaiting () {
        // Reset the flag for each address        
        for(uint8 i=0; i<whitelist.length; i++){
           waiting[ whitelist[i] ] = true;
        }
    }
}

"""

In [10]:
compiled['<stdin>:aggregator']['abi']

[{u'constant': True,
  u'inputs': [],
  u'name': u'iteration',
  u'outputs': [{u'name': u'', u'type': u'uint8'}],
  u'payable': False,
  u'type': u'function'},
 {u'constant': False,
  u'inputs': [],
  u'name': u'resetWaiting',
  u'outputs': [],
  u'payable': False,
  u'type': u'function'},
 {u'constant': True,
  u'inputs': [{u'name': u'', u'type': u'uint256'}],
  u'name': u'whitelist',
  u'outputs': [{u'name': u'', u'type': u'address'}],
  u'payable': False,
  u'type': u'function'},
 {u'constant': True,
  u'inputs': [{u'name': u'', u'type': u'address'}],
  u'name': u'waiting',
  u'outputs': [{u'name': u'', u'type': u'bool'}],
  u'payable': False,
  u'type': u'function'},
 {u'constant': False,
  u'inputs': [],
  u'name': u'stillWaiting',
  u'outputs': [{u'name': u'', u'type': u'bool'}],
  u'payable': False,
  u'type': u'function'},
 {u'inputs': [{u'name': u'_whitelist', u'type': u'address[]'}],
  u'payable': False,
  u'type': u'constructor'}]

In [12]:
# Basic contract compiling process.
#   Requires that the creating account be unlocked.
#   Note that by default, the account will only be unlocked for 5 minutes (300s). 
#   Specify a different duration in the geth personal.unlockAccount('acct','passwd',300) call, or 0 for no limit

compiled = compile_source(source)
compiledCode = compiled['<stdin>:aggregator']['bin']
compiledCode = '0x'+compiledCode # This is a hack which makes the system work

addressList = [x[2:] for x in c.eth_accounts()]
contractTx = c.create_contract(c.eth_coinbase(), compiledCode, gas=3000000,sig='aggregator(address[])',args=[addressList])
# contractTx = c.create_contract(c.eth_coinbase(), compiledCode, gas=3000000,sig='greeter(string)',args=['Hello World!'])
print("Contract transaction id is "+contractTx)

print("Waiting for the contract to be mined into the blockchain...")
while c.eth_getTransactionReceipt(contractTx) is None:
    time.sleep(1)

contractAddr = c.get_contract_address(contractTx)
print("Contract address is "+contractAddr)

Contract transaction id is 0x50f15255ba7079c5c53ef6e4214428564c24c4f84f2c2faa2485d29e276d3194
Waiting for the contract to be mined into the blockchain...
Contract address is 0x884446527421b7a178c42d12620fa9bc98238442


In [45]:
c.call('0x3f211b2256bc7b64f365cbc7bbff4ae77e22a151','indexer()',[],['int8'])

[3]

In [55]:
c.call('0x3f211b2256bc7b64f365cbc7bbff4ae77e22a151','names(uint256)',[2],['uint256'])

[350]

# Other Stuff
Stackoverflow question at https://stackoverflow.com/questions/44373531/contract-method-not-returning-the-value-while-using-ethjsonrpc-and-pyethapp

In [132]:
source = """pragma solidity ^0.4.2;
contract Example {
    // Device Registry
    mapping (uint => string) public _registry;
    uint nIndex= 0; 
    function set_s(string new_s) {
        _registry[nIndex] = new_s;
        nIndex = nIndex + 1;
    }
    function get_s(uint number) returns (string) {
        return _registry[number];
    } 
}
"""

compiled = compile_source(source)
# compiled = compile_files(['Solidity/ethjsonrpc_tutorial.sol'])  #Note: Use this to compile from a file
compiledCode = compiled['Example']['bin']
compiledCode = '0x'+compiledCode # This is a hack which makes the system work

# Put the contract in the pool for mining, with a gas reward for processing
contractTx = c.create_contract(c.eth_coinbase(), compiledCode, gas=3000000)
print("Contract transaction id is "+contractTx)

print("Waiting for the contract to be mined into the blockchain...")
while c.eth_getTransactionReceipt(contractTx) is None:
    time.sleep(1)

contractAddr = c.get_contract_address(contractTx)
print("Contract address is "+contractAddr)

OSError: [Errno 2] No such file or directory

In [110]:
tx = c.call_with_transaction(c.eth_coinbase(), contractAddr, 'set_s(string)', ['Dinesh'])
while c.eth_getTransactionReceipt(tx) is None:
    time.sleep(1)